In [1]:
import torch
import numpy as np
import pandas as pd
from scipy.stats.contingency import relative_risk
from math import isnan
import json



In [2]:
def risk_reward_fn(vec):
    # Determined by polypharmacy definition
    if vec.sum() < 5:
        return 0

    vec_indices = torch.where(vec == 1)[0]

    # Exposed
    rows_exposed = torch.where((X[:, vec_indices] == 1).all(axis=1))[
        0
    ]
    rows_control = torch.where((X[:, vec_indices] == 0).any(axis=1))[
        0
    ]

    rows_exposed_case = torch.where(y[rows_exposed] == 1)[0]
    rows_control_case = torch.where(y[rows_control] == 1)[0]

    n_exposed = len(rows_exposed)
    n_exposed_case = len(rows_exposed_case)
    n_control = len(rows_control)
    n_control_case = len(rows_control_case)
    rr = relative_risk(n_exposed_case, n_exposed, n_control_case, n_control).relative_risk
    
    if isnan(rr):
        # Interpreted as 0 by experts
        return 0

    elif rr == float('inf'):
        return 10   # Return something really big, but not inf so it doesn't break the regression

    return rr

In [3]:
min_risks = []
max_risks = []
avg_n_rx = []
ds = [25, 150, 550, 1600]
for n_dim in ds:
    for run_number in range(50):
        print(f"{n_dim}: {run_number}")
        df = pd.read_csv(f"datasets/polypharmacie/10000r_{n_dim}c_1o_run{run_number}.csv")

        with open(f"datasets/polypharmacie/10000r_{n_dim}c_1o_run{run_number}_config.json", "r") as file:
            config = json.load(file)

        with open(f"datasets/polypharmacie/10000r_{n_dim}c_1o_run{run_number}.json", "r") as file:
            pattern_config = json.load(file)

        pattern_codes = [
            np.array(pattern_config[f"pattern_{i}"]["code_indices"]) - 1 for i in range(10)
        ]

        n_medical_codes = config["n_medical_codes"]
        n_outcomes = config["n_outcomes"]
        column_names = (
            ["patient_id"]
            + [f"medical_code_{i}" for i in range(n_medical_codes)]
            + [f"outcome_code_{i}" for i in range(n_outcomes)]
        )

        df.columns = column_names

        X_df = df.iloc[:, 1 : n_medical_codes + 1]
        X = X_df.values

        y_df = df.iloc[:, n_medical_codes + 1 : n_medical_codes + 1 + n_outcomes]
        y = y_df.values.ravel()

        X_df.describe()
        X = torch.Tensor(X)
        y = torch.Tensor(y).cpu()
        set_existing_vecs = torch.unique(X, dim=0).float()


        risks = []
        epsilon = 0.4
        inv_eps = 1 - epsilon
        for vec in set_existing_vecs:
            risks.append(risk_reward_fn(vec))

        risks = np.array(risks)

        max_risk = max(risks)
        min_risk = min(risks)
        max_risks.append(max_risk)
        min_risks.append(min_risk)

        n_rx_in_dataset = 0
        for row in X:
            n_rx_in_dataset += row.sum()

        avg_n_rx.append(n_rx_in_dataset / len(X))

25: 0
25: 1
25: 2
25: 3
25: 4
25: 5
25: 6
25: 7
25: 8
25: 9
25: 10
25: 11
25: 12
25: 13
25: 14
25: 15
25: 16
25: 17
25: 18
25: 19
25: 20
25: 21
25: 22
25: 23
25: 24
25: 25
25: 26
25: 27
25: 28
25: 29
25: 30
25: 31
25: 32
25: 33
25: 34
25: 35
25: 36
25: 37
25: 38
25: 39
25: 40
25: 41
25: 42
25: 43
25: 44
25: 45
25: 46
25: 47
25: 48
25: 49
150: 0
150: 1
150: 2
150: 3
150: 4
150: 5
150: 6
150: 7
150: 8
150: 9
150: 10
150: 11
150: 12
150: 13
150: 14
150: 15
150: 16
150: 17
150: 18
150: 19
150: 20
150: 21
150: 22
150: 23
150: 24
150: 25
150: 26
150: 27
150: 28
150: 29
150: 30
150: 31
150: 32
150: 33
150: 34
150: 35
150: 36
150: 37
150: 38
150: 39
150: 40
150: 41
150: 42
150: 43
150: 44
150: 45
150: 46
150: 47
150: 48
150: 49
550: 0
550: 1
550: 2
550: 3
550: 4
550: 5
550: 6
550: 7
550: 8
550: 9
550: 10
550: 11
550: 12
550: 13
550: 14
550: 15
550: 16
550: 17
550: 18
550: 19
550: 20
550: 21
550: 22
550: 23
550: 24
550: 25
550: 26
550: 27
550: 28
550: 29
550: 30
550: 31
550: 32
550: 33
550: 34


In [4]:
print("min and max avg_n_rx")
print(np.min(avg_n_rx))
print(np.max(avg_n_rx))

print("min and max min_risks")
print(np.min(min_risks))
print(np.max(min_risks))

print("min and max max_risks")
print(np.min(max_risks))
print(np.max(max_risks))

min and max avg_n_rx
6.189119
12.693469
min and max min_risks
0.0
0.0
min and max max_risks
1.7732434350603266
3.7899127796738723
